# RNA-seq Exploratory Analysis

This notebook demonstrates exploratory data analysis of RNA-seq results.

**OutWardly Omics & Bioinformatics Demo**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import duckdb

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

%matplotlib inline

## 1. Load Data from Database

In [ ]:
# Connect to DuckDB
conn = duckdb.connect('../results/metadata.db', read_only=True)

# Load DESeq2 results
deseq_results = conn.execute('SELECT * FROM deseq2_results').fetchdf()

# Load significant genes
sig_genes = conn.execute('SELECT * FROM significant_genes').fetchdf()

# Load summary
summary = conn.execute('SELECT * FROM analysis_summary').fetchdf()

print('Data loaded successfully!')
print(f'Total genes: {len(deseq_results)}')
print(f'Significant genes: {len(sig_genes)}')

## 2. Summary Statistics

In [ ]:
print('Analysis Summary:')
print('=' * 60)
print(summary.to_string(index=False))

# Distribution of p-values
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# P-value histogram
axes[0].hist(deseq_results['pvalue'].dropna(), bins=50, edgecolor='black')
axes[0].set_xlabel('P-value')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of P-values')

# Log2 fold change distribution
axes[1].hist(deseq_results['log2FoldChange'].dropna(), bins=50, edgecolor='black')
axes[1].set_xlabel('log2 Fold Change')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of log2 Fold Changes')

plt.tight_layout()
plt.show()

## 3. MA Plot

In [ ]:
# MA plot (log2FC vs mean expression)
fig, ax = plt.subplots(figsize=(12, 8))

# Non-significant genes
ns = deseq_results[deseq_results['padj'] >= 0.05]
ax.scatter(ns['baseMean'], ns['log2FoldChange'], 
           c='lightgray', alpha=0.3, s=10, label='Not significant')

# Significant genes
sig = deseq_results[deseq_results['padj'] < 0.05]
ax.scatter(sig['baseMean'], sig['log2FoldChange'], 
           c='red', alpha=0.6, s=20, label='Significant (padj < 0.05)')

ax.set_xscale('log')
ax.axhline(y=0, color='blue', linestyle='--', linewidth=1)
ax.set_xlabel('Mean Expression (log scale)', fontsize=12)
ax.set_ylabel('log2 Fold Change', fontsize=12)
ax.set_title('MA Plot: Treated vs Control', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Top Differentially Expressed Genes

In [ ]:
# Top 20 upregulated genes
top_up = sig_genes[sig_genes['log2FoldChange'] > 0].nsmallest(20, 'padj')
print('Top 20 Upregulated Genes:')
print('=' * 80)
print(top_up[['gene_id', 'log2FoldChange', 'padj']].to_string(index=False))

print('\n')

# Top 20 downregulated genes
top_down = sig_genes[sig_genes['log2FoldChange'] < 0].nsmallest(20, 'padj')
print('Top 20 Downregulated Genes:')
print('=' * 80)
print(top_down[['gene_id', 'log2FoldChange', 'padj']].to_string(index=False))

## 5. Gene Set Enrichment Preview

This section would typically include pathway analysis using tools like:
- Gene Ontology (GO) enrichment
- KEGG pathway analysis
- Reactome pathways
- MSigDB gene sets

In [ ]:
# Example: Extract gene lists for downstream enrichment analysis
upregulated_genes = sig_genes[sig_genes['log2FoldChange'] > 0]['gene_id'].tolist()
downregulated_genes = sig_genes[sig_genes['log2FoldChange'] < 0]['gene_id'].tolist()

print(f'Upregulated genes: {len(upregulated_genes)}')
print(f'Downregulated genes: {len(downregulated_genes)}')

# Save gene lists for enrichment analysis
with open('../results/upregulated_genes.txt', 'w') as f:
    f.write('\n'.join(upregulated_genes))

with open('../results/downregulated_genes.txt', 'w') as f:
    f.write('\n'.join(downregulated_genes))

print('\nGene lists saved for enrichment analysis!')

## 6. Database Queries

Examples of useful SQL queries on the RNA-seq database

In [ ]:
# Query: Top 10 genes by absolute fold change
query = """
SELECT gene_id, log2FoldChange, padj
FROM deseq2_results
WHERE padj < 0.05
ORDER BY ABS(log2FoldChange) DESC
LIMIT 10
"""

top_abs_fc = conn.execute(query).fetchdf()
print('Top 10 genes by absolute fold change:')
print(top_abs_fc.to_string(index=False))

In [ ]:
# Query: Genes with high significance but moderate fold change
query = """
SELECT gene_id, log2FoldChange, padj, baseMean
FROM deseq2_results
WHERE padj < 0.001
  AND ABS(log2FoldChange) BETWEEN 0.5 AND 1.5
  AND baseMean > 100
ORDER BY padj
LIMIT 20
"""

moderate_fc = conn.execute(query).fetchdf()
print('Highly significant genes with moderate fold change:')
print(moderate_fc.to_string(index=False))

## Summary

This notebook demonstrated:
1. Loading RNA-seq results from DuckDB
2. Summary statistics and quality checks
3. MA plots and volcano plots
4. Identification of top differentially expressed genes
5. SQL queries for flexible data exploration

**Next steps:**
- Functional enrichment analysis
- Network analysis
- Integration with other omics data
- Machine learning predictions

---

**OutWardly** | Omics & Bioinformatics Services

Contact: hello@outwardly.net

In [ ]:
# Close database connection
conn.close()
print('Analysis complete!')